In [2]:
PATH = "/home/dude/Desktop/data/network_data/"

In [5]:
import numpy as np
import pandas as pd
import os

In [8]:
df1 = pd.read_csv(PATH + 'Dataset_1.csv')

In [9]:
df1

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,108.217.33.201,18.160.225.27,TLSv1.2,801,Application Data
1,2,0.009384,18.160.225.27,108.217.33.201,TLSv1.2,733,Application Data
2,3,0.009465,18.160.225.27,108.217.33.201,TCP,1494,443 > 23117 [ACK] Seq=680 Ack=748 Win=513 Le...
3,4,0.009478,18.160.225.27,108.217.33.201,TCP,1494,443 > 23117 [ACK] Seq=2120 Ack=748 Win=513 L...
4,5,0.009552,18.160.225.27,108.217.33.201,TCP,1494,443 > 23117 [ACK] Seq=3560 Ack=748 Win=513 L...
...,...,...,...,...,...,...,...
493,496,0.015173,18.160.225.27,108.217.33.201,TCP,1494,443 > 23117 [ACK] Seq=670585 Ack=748 Win=513...
494,497,0.015175,18.160.225.27,108.217.33.201,TCP,1494,443 > 23117 [ACK] Seq=672025 Ack=748 Win=513...
495,498,0.015178,18.160.225.27,108.217.33.201,TCP,1494,443 > 23117 [ACK] Seq=673465 Ack=748 Win=513...
496,499,0.015180,18.160.225.27,108.217.33.201,TLSv1.2,712,Application Data
